In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('homework_6.1.csv', index_col=0)

In [6]:
df.head()

,Z,X,Y
0,0.548814,0,-0.823220
1,0.715189,1,0.842405
2,0.602763,1,0.898618
3,0.544883,0,-0.817325
4,0.423655,0,-0.635482


In [7]:
from sklearn.neighbors import NearestNeighbors

# Separate data into treated and untreated groups
treated = df[df['X'] == 1]
untreated = df[df['X'] == 0]

# Reshape Z for NearestNeighbors
treated_Z = treated['Z'].values.reshape(-1, 1)
untreated_Z = untreated['Z'].values.reshape(-1, 1)

# For ATT: Match each treated individual to nearest untreated
nn_untreated = NearestNeighbors(n_neighbors=1).fit(untreated_Z)
distances_treated_to_untreated, indices_treated_to_untreated = nn_untreated.kneighbors(treated_Z)
treated_counterfactuals = untreated.iloc[indices_treated_to_untreated.flatten()]['Y'].values
att_effects = treated['Y'].values - treated_counterfactuals
att = att_effects.mean()

# For ATUT: Match each untreated individual to nearest treated
nn_treated = NearestNeighbors(n_neighbors=1).fit(treated_Z)
distances_untreated_to_treated, indices_untreated_to_treated = nn_treated.kneighbors(untreated_Z)
untreated_counterfactuals = treated.iloc[indices_untreated_to_treated.flatten()]['Y'].values
atut_effects = untreated_counterfactuals - untreated['Y'].values
atut = atut_effects.mean()

# For ATE: Weighted average of ATT and ATUT
n_treated = len(treated)
n_untreated = len(untreated)
n_total = n_treated + n_untreated
ate = (n_treated / n_total) * att + (n_untreated / n_total) * atut

# For MTE: Maximum treatment effect across all untreated items
mte = atut_effects.max()

print(f"Average Treatment Effect (ATE): {ate:.6f}")
print(f"Average Treatment Effect on the Treated (ATT): {att:.6f}")
print(f"Average Treatment Effect on the Untreated (ATUT): {atut:.6f}")
print(f"Marginal Treatment Effect (MTE): {mte:.6f}")

Average Treatment Effect (ATE): 1.695270
Average Treatment Effect on the Treated (ATT): 1.846409
Average Treatment Effect on the Untreated (ATUT): 1.549477
Marginal Treatment Effect (MTE): 2.172470
